In [1]:
# Necessary packages
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import cv2
import time
import pickle
import random
from tqdm import tqdm
from astropy.io import fits
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D

In [3]:
# CNN Model
X = pickle.load(open("X.pickle", "rb"))
Y = pickle.load(open("Y.pickle", "rb"))

conv_layers = [1,2,3,4,5]
layer_sizes = [2,4,8,16,32,64]

# setup to test optimal parameters
for conv_layer in conv_layers:
    for layer_size in layer_sizes:
        NAME = "{}-conv-{}-nodes-".format(conv_layer, layer_size, int(time.time()))
        print(NAME)

        model = Sequential()

        for l in range(conv_layer):
            model.add(Conv1D(kernel_size=layer_size,
                             filters=32,
                             input_shape=X.shape[1:],
                             activation='relu'))
            model.add(Conv1D(kernel_size=layer_size,
                             filters=16,
                             activation='relu'))
            model.add(MaxPooling1D(pool_size=4))

        model.add(Flatten())
        model.add(Dense(3))
        model.add(Activation('softmax'))

        # used to see visual graph of optimization for each iteration
        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        model.fit(X, Y,
                  batch_size=25,
                  epochs=15,
                  validation_split=0.2,
                  callbacks=[tensorboard])

1-conv-2-nodes-


AttributeError: 'list' object has no attribute 'shape'

In [4]:
# Save the best model for predictions
model.save(NAME)